## Classifying visual perception from scalp EEG
* We will work with a dataset where subjects observed images of target and non-target objects while having their neural activity recorded with scalp EEG
* We will create and fit a model that predicts which category of object they saw based on their observed brain activity
* This takes the form of a binary classification problem. 
* The model that we will use is known as a "convolutional" network, due to it transforming the input data in a manner analogous to filtering in linear systems

### Loading the data
* The data is contained in a Matlab data file
* We use the scipy package to conveniently bring it into the Python workspace

In [ ]:
filename = '../data/EEG.mat' # the data file
import scipy.io as sio # the library that we use to load Matlab data into Python
data = sio.loadmat(filename)

### Inspect the data
* The data has been partitioned into a training and test set
* Features are shaped (example, electrode, time samples)
* Labels are shaped (example, 2), where the 2 refers to the two classes (target vs non-target)

In [ ]:
x_train = data['X_train']
x_test = data['X_test']
y_train = data['Y_train']
y_test = data['Y_test']

print("The shape of the training features is " + str(x_train.shape))
print("The shape of the training labels is " + str(y_train.shape))
print("The shape of the test features is " + str(x_test.shape))
print("The shape of the test labels is " + str(y_test.shape))

### Visualizing a sample trial
* Looking at the data is always recommended
* Here the data takes the form of space-time matrix, that we plot as overlaid waveforms

In [ ]:
import numpy as np # Python's array library
import matplotlib.pyplot as plt # a popular library for plotting in Python
trial_idx = 500 # let's look at trial 501
sample_eeg = np.squeeze(x_train[trial_idx,:,:])
plt.plot(sample_eeg.T)
plt.show()

### Inspect the label information
* It is common to label each example with a vector of probabilities
* Below, the first element of each vector denotes P(non-target)
* Notice above that all but one of the first 10 examples was a non-target. 
    * The 7th example was a target.

In [ ]:
print(y_train[0:10,:])

### Creating the model
* We use the user-friendly Keras library to create a simple model for predicting target vs non-target based on the array of EEG values
* The first layer of our model corresponds to the input data
* The second layer performs a convolution (in effect spatial filtering) across the data in windows of 10 samples each
* The final layer transforms the output of the convolution into a probability value
* Our model has 881 parameters. We have 1058 examples. 
    * It's a good rule-of-thumb to have at least as many training examples as model parameters. 

In [ ]:
import keras
input_shape=(64,128,1) # this tells Keras the shape of the data that will be passed to it for training/testing
num_classes=2
num_filters = 1
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(num_filters, kernel_size=(64, 10), activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

### Compile the model
* We need to tell Keras what type of loss function we will be using, and which optimizer we would like. 
* The batch size controls how many examples are used for each computation of the gradient of the loss function
* The number of epochs controls how many complete passes of the data we would like to make
* We also tell Keras that we want to monitor both the model accuracy, as well as the area under the ROC curve.

In [ ]:
batch_size = 128  # how many training examples in each "batch" used to compute gradient of loss function
epochs = 10 # how many passes through the data we want the fitting to take

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])

### Fitting the model
* Keras allows you to monitor the training performance during model fitting
* Notice that the training performance is quite good
    * Area under the ROC curve of is ~ 0.87
    * Now let's see how well the model generalizes to unseen data

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

### Evaluate the model on unseen data.
* We use the test set to assess our model performance

In [ ]:
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1])) # element 0 is loss, 1 is accuracy, 2 is auroc
print("The model AUROC is " + str(metrics[2]))

### Verify performance manually
* We can use the predict function to generate model probabilities for the examples in the test set
* A probability > 0.5 should be interpreted as the model predicting a target image
* In the "pred_labels" variable below, the first column is P(non-target), while the second is P(target).
* In the "true_labels" variable, the binary value indicates the ground truh (0 for non-target, 1 for target).

In [ ]:
test_scores = model.predict(x_test)
pred_labels = np.argmax(test_scores,1)
true_labels = np.argmax(y_test,1)
test_accuracy = np.mean(true_labels==pred_labels)

print("Predicted probabilities:")
print(test_scores[0:10,:])

print("Test set labels:")
print(pred_labels[0:10])


print("Accuracy = " + str(test_accuracy))

## Increasing the model complexity
* We can increase the number of convolutional "filters" in our learning model
* We can think of each filter as looking for a certain pattern in the EEG
* If more than one distinct pattern reflects perception of an oddball, this could improve detection
* We now have 3 times as many parameters. 
    * This puts us at risk of overfitting, but we proceed.

In [ ]:
num_filters = 3
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(num_filters, kernel_size=(64, 10), activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

## Compiling and fitting the new model
* We fit the model again
* Due to the increase in the number of model parameters, the training performance is guaranteed to improve

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

## Evaluating the new model on unseen data
* We are now ready to examine the model's performance on the test set
* Indeed, we see a marked improvement on the AUROC
    * The increased model complexity did not hurt us in this case

In [ ]:
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1]))
print("The model AUROC is " + str(metrics[2]))

## Adding regularization
* Keras makes it easy to add both L1 and L2 regularization
* We will add both forms to our model
* Here we assign fairly arbitrary values to the so-called "hyperparameters"
    * In practice, these have to be extensively tuned on a validation set

In [ ]:
reg_l1_l2 = keras.regularizers.l1_l2(l1=0.001, l2=0.001)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(num_filters, kernel_size=(64, 10), activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer = reg_l1_l2),
    ]
)

model.summary()

## Compiling and fitting the model
* Once again, we need to compile and fit the hopefully improved model
* With regularization, we expect the training accuracy to suffer, but hopefully also lead to improved generalization

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1]))
print("The model AUROC is " + str(metrics[2]))

## Unequal class prevalence
* Targets are infrequent in P300 experiments (by design)
* This class imbalance may lead to the model fitting emphasizing the more frequent non-target class
* First we need to measure the proportion of the two classes in the _training_ set

In [ ]:
train_labels = np.argmax(y_train,1)
n_non_targets = np.sum(train_labels==0)
n_targets=np.sum(train_labels==1)
class_ratio = n_targets/n_non_targets
print("The ratio of targets to non targets is " + str(class_ratio))

## Adding class weighting
* Now we add class weighting to balance the contribution of the classes to the loss function
* This option is included in Keras fit function
* After fitting the model with the class weights, we evaluate performance one last time

In [ ]:
class_weight={0:1,1:1/class_ratio}

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weight)
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1]))
print("The model AUROC is " + str(metrics[2]))